In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/income-dataset-itu-project/income_dataset_ITU_project.csv")

DATA SET

Projede kullanılan dataset UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Adult)'den elde edilmiştir. Data Set 14 attributes halinde insanların demografik bilgilerinden oluşmaktadır.

-age : Yaş

-workclass : Çalışma şekli.Attributes: Federal-gov, Local-gov, Never-worked, Private, Self-emp-inc, Self-emp-not-inc, State-gov, and Without-pay.

-education : Eğitim seviyesi.Sıralanmış eğtim seviyeri attibutes: Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate.

-education_num : Eğitim seviyelerinin nümerik formu

-marital_status : Evlilik durumu. Evlilik durumu attributes: Divorced, Married-AF-spouse, Married-civ-spouse, Married-spouse-absent, Never-married, Separated, and Widowed.

-occupation : İşi.İş attributes: Adm-clerical, Armed-Forces, Craft-repair, Exec-managerial, Farming-fishing, Handlers-cleaners, Machine-op-inspct, Other-service, Priv-house-serv, Prof-specialty, Protective-serv, Sales, Tech-support, and Transport-moving.

-relationship : Aile durumu.Aile durumu attributes: Husband, Not-in-family, Other-relative, Own-child, Unmarried, and Wife.

-race : Irkı.Irk attributes: Amer-Indian-Eskimo, Asian-Pac-Islander, Black, Other, and White.

-sex : Cinsiyeti.Cinsiyet attribute. The levels of the attributes: Female and Male.

-capital_gain : Değer artışı.

-capital_loss : Değer azaışı.

-hours_per_week : Haftalık çalışma saati.

-native_country – Menşei.Menşei attributes: Cambodia, Canada, China, Columbia, Cuba, Dominican-Republic, Ecuador, El-Salvador, England, France, Germany, Greece, Guatemala, Haiti, Holand-Netherlands, Honduras, Hong, Hungary, India, Iran, Ireland, Italy, Jamaica, Japan, Laos, Mexico, Nicaragua, Outlying-US(Guam-USVI-etc), Peru, Philippines, Poland, Portugal, Puerto-Rico, Scotland, South, Taiwan, Thailand, Trinadad&Tobago, United-States, Vietnam,Yugoslavia.

-income –Yıldaki $50000 kazanım durum.Attributes: <=50K-0 ve >50K-1.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


Datanın Boyutu (32561 satır ve 14 kolondan oluşmaktadır)

In [ ]:
data.shape

Datanın Kolonları

In [ ]:
data.columns

Datanın Kolonlarında işlem kolaylığı olması açısından kolon isimleri değiştirilmesi

In [ ]:
data.columns=['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital-gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income']
data.columns

Pandas kütüphanesi ile DataFrame formatına dönüştürme ve ilk 20 datayı okuma

In [ ]:
data=pd.DataFrame(data)
data.head(20)

Datamızda bulunan nümerik dataların istatiski olarak gösterimi (maksimum,minumum,ortalama,medyan vb.)

In [ ]:
data.describe()

Öncellikle kullanılan data da boş (null) olan kolonlardaki sayıların toplamı çıkartıldı

In [ ]:
data.isnull().sum().sum()

Hangi kolonda kaç tane boş datamızın olduğu çıkartıldı.

In [ ]:
data.isnull().sum()

'workclass','native_country','occupation' kolonlarında boş data olduğu görüldü.Burada bulunan null datalar drop edilmesi model uygulamada hatalı sonuçlar vereceği değerlendiğinden, 'Unknown' olarak dolduruldu. 

In [ ]:
data["native_country"]=data["native_country"].fillna("Unknown")

In [ ]:
data["workclass"]=data["workclass"].fillna("Unknown")

In [ ]:
data["occupation"]=data["occupation"].fillna("Unknown")

Datamızdaki kolonlarımızın birbirleri ile olan ilişkilerini gözlemek için korelasyonları çıkartıldı.

In [ ]:
data.corr()

Korelasyonu daha net görebilmek adına seaborn kütüphanesi ile görselliği artırıldı.Korelasyon incelendiğnde 'education_num' ile 'income' arasında en fazla pozitif bir korelasyon olduğu gözlemlendi.

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(),annot=True,linewidth=.5,fmt='.2f',ax=ax)
data.corr()
plt.show()

DATA GÖRSELLEŞTİRME

Datada bulunan meslek gruplarından ki dağılımı seaborn kütüphanesi ile görselleştirme.En fazla meslek grubu 'Prof-Specialty' grubunda olduğu gözlemlenmiştir.

In [ ]:
f,ax=plt.subplots(figsize=(20,20))
sns.countplot(data['occupation'],label="Toplam")
plt.show()

Aynı mantıkla datamızda bulunan eğitim seviyeleri görsellenmiştir.En fazla 'HS-grad' eğitim seviyesinde kişi buunmaktadır.

In [ ]:
f,ax=plt.subplots(figsize=(20,20))
sns.countplot(data['education'],label="Toplam")
plt.show()

Eğitim seviyesine göre 50K yıllık kazanç durumu görsellenmiştir. En fazla kişi 'HS-grad' olmasına rağmen dataya bakıldığı zaman en fazla kazanç 'Doctorate' eğitim seviyesindeki kişilerde gözlenmektedir.

In [ ]:
f,ax=plt.subplots(figsize=(20,20))
g = sns.barplot(x="education",y="income",data=data)
g = g.set_ylabel("50K$")
plt.show()

Toplamda kişilerin kazançlarının görselleştirilmesi.

In [ ]:
sns.countplot(data['income'],label="Count")
plt.show()

Cinsiyetlere göre yıllık 50K kazanma görselleştirmesi. 

In [ ]:
g = sns.barplot(x="sex",y="income",data=data)
g = g.set_ylabel("50K$")
plt.show()

FEATURE ENGİNEERİNG

Nümerik olamayan "education","native_country","marital_status","sex","workclass","occupation","race" kolonlarında kullanılan veriler gözlenmiştir.

In [ ]:
data_edu=data["education"]
data_edu.unique()

In [ ]:
data_country=data["native_country"]
data_country.unique()

In [ ]:
data_marital=data["marital_status"]
data_marital.unique()

In [ ]:
data_sex=data["sex"]
data_sex.unique()

In [ ]:
data_work=data["workclass"]
data_work.unique()

In [ ]:
data_occupation=data["occupation"]
data_occupation.unique()

In [ ]:
data_race=data["race"]
data_race.unique()

"sex" kolonu Male ve Female olarak nümerik formata döndürülmüştür. 

In [ ]:
data["sex"] = data["sex"].map({"Male": 0, "Female":1})

"martial_status" kolonu Single ve Married olarak nümerik formata döndürülmüştür.

In [ ]:
data["marital_status"] = data["marital_status"].replace(['Never-married','Divorced','Separated','Widowed'], 'Single')
data["marital_status"] = data["marital_status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'], 'Married')
data["marital_status"] = data["marital_status"].map({"Married":1, "Single":0})
data["marital_status"] = data["marital_status"].astype(int)

"education" kolonu HighEdu ve LowEdu olarak nümerik formata döndürülmüştür.

In [ ]:
data["education"] = data["education"].replace(["Bachelors","Masters","Doctorate","Prof-school"],"HighEdu")
data["education"] = data["education"].replace(["Preschool","1st-4th","5th-6th","7th-8th","9th","10th","11th","12th","HS-grad", 
                    "Assoc-acdm","Assoc-voc","Some-college"],"LowEdu")
data["education"] = data["education"].map({"HighEdu":1, "LowEdu":0})
data["education"] = data["education"].astype(int)


"race" kolonu WhiteRace ve OtherRace olarak nümerik formata döndürülmüştür.

In [ ]:
data["race"] = data["race"].replace(['White'],"WhiteRace")
data["race"] = data["race"].replace(['Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo','Other'],"OtherRace")
data["race"] = data["race"].map({"WhiteRace":1, "OtherRace":0})
data["race"] = data["race"].astype(int)

"native_country" kolonu HighPriceCounty ve LowPriceCountry olarak nümerik formata döndürülmüştür.

In [ ]:
data["native_country"] = data["native_country"].replace(['United-States','England','Canada','Germany', 'France', 'Italy',
                        'China', 'Japan', 'Scotland', 'Ireland','Holand-Netherlands'],"HighPriceCountry")

data["native_country"] = data["native_country"].replace(['Cuba', 'Jamaica', 'India', 'Unknown', 'Mexico',
                        'South', 'Puerto-Rico', 'Honduras','Iran', 'Philippines', 'Poland','Columbia', 'Cambodia',
                        'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal','Yugoslavia', 'Peru',
                        'Outlying-US(Guam-USVI-etc)', 'Trinadad&Tobago','Greece', 'Nicaragua', 'Vietnam', 'Hong',
                        'Hungary','Dominican-Republic', 'El-Salvador', 'Guatemala' ],"LowPriceCountry")
data["native_country"] = data["native_country"].map({"HighPriceCountry":1, "LowPriceCountry":0})
data["native_country"] = data["native_country"].astype(int)

Nümerik Formata döndürülen data.

In [ ]:
data.head(20)

Modellemede kullanılmayacak nümerik formatta olmayan dataların setten atılması

In [ ]:
data.drop(labels=["workclass","occupation","relationship"], axis = 1, inplace = True)
data.head(20)

Kullanılacak olan ve nümerik formata dönüştürülen datanın korelasyon matrisi.

In [ ]:
f,ax=plt.subplots(figsize=(18,18))
sns.heatmap(data.corr(),annot=True,linewidth=.5,fmt='.2f',ax=ax)
data.corr()
plt.show()

Bu aşamada data girdier ve çıktılar olarak ikiye bölünecek girdi olarak kullanılan verilerle çıktıyı modeller kullanılarak tahminlemeye çalışılacaktır.


In [ ]:
data

MODELLEME

Data girdiler ve çıktı olarak ayrılır.Girdiler ve çıktı olarak ayrılan datalar eğitim ve test olarak belirlenir.Burada test edilecek datalar %20 olarak ayrılmıştır.Modellemeden önce yapılacak işlem train ve test datası boyutlarını ayrıştırmaktır.


In [ ]:
X= data.iloc[:,:10]
y=data.iloc[:,10]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=True)

a)KNN Classification

KNN modellinde kullanaılacak parametreleri belirlemek için GridSearchCV ile en iyi score'u üretecek parametreler bulunur.

In [ ]:
param_grid = {'n_neighbors':[4,5,6,7],'leaf_size':[1,3,5],'algorithm':['auto', 'kd_tree'],'n_jobs':[-1]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=5)
grid.fit(X, y)
grid.best_params_

GridsearchCV ile bulunan en iyi parametreler KNN modeline parametre olarak uygulanr.Sınıflandırma raporlanır ve accuracy hesaplanır.

In [ ]:
knn = KNeighborsClassifier(algorithm="auto",leaf_size=1,n_jobs=-1,n_neighbors=6)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
knn.score(X_test,y_test)

Cross-validation ile makine öğrenmesi modelinin görmediği veriler üzerindeki performansını ölçmek amaçlı kullanılır.Knn modeli için Cross Validation Score Hesalandı ve Accuracy ile karşılaşırıldı.

In [ ]:

clf =KNeighborsClassifier(algorithm="auto",leaf_size=1,n_jobs=-1,n_neighbors=6)
print(cross_val_score(clf, X, y, cv=10, n_jobs=1))

b)Logistic Regression

Lojistic Regression modellinde kullanaılacak parametreleri belirlemek için GridSearchCV ile en iyi score'u üretecek parametreler bulunur.

In [ ]:
param_grid = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[0.001, 0.01, 0.1, 1, 10, 100],
             'solver':['lbfgs', 'liblinear'], 'l1_ratio':[0.001, 0.01, 0.1]}

grid = GridSearchCV(LogisticRegression(), param_grid=param_grid, verbose=3)

grid.fit(X, y)
grid.best_params_

GridsearchCV ile bulunan en iyi parametreler Logistic Regression modeline parametre olarak uygulanr.Sınıflandırma raporlanır ve accuracy hesaplanır.

In [ ]:
logmodel = LogisticRegression(C= 0.01, l1_ratio= 0.1, penalty= 'l1', solver= 'liblinear',max_iter=1000)
logmodel.fit(X_train,y_train)
y_pred = logmodel.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
logmodel.score(X_test,y_test)

Logistic Regression modeli için Cross Validation Score Hesalandı ve Accuracy ile karşılaşırıldı.

In [ ]:
clf =LogisticRegression(C= 0.01, l1_ratio= 0.1, penalty= 'l1', solver= 'liblinear',max_iter=1000)
print(cross_val_score(clf, X, y, cv=10, n_jobs=1))

c)Random Forest Classifier

Random Forest Classifier modellinde kullanaılacak parametreleri belirlemek için GridSearchCV ile en iyi score'u üretecek parametreler bulunur.

In [ ]:
param_grid = {'criterion':['gini', 'entropy'], 'max_depth':[5,10,15,20], 'n_estimators':[500,600,700,800,900,1000]}
grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, verbose=3)

grid.fit(X, y)
grid.best_params_

GridsearchCV ile bulunan en iyi parametreler Random Forest Classifier modeline parametre olarak uygulanr.Sınıflandırma raporlanır ve accuracy hesaplanır.

In [ ]:
randomforest = RandomForestClassifier(max_depth=15,random_state=42,criterion="entropy",n_estimators=900)
randomforest.fit(X_train, y_train)
y_pred = randomforest.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
randomforest.score(X_test,y_test)

RandomForestClassifier modeli için Cross Validation Score Hesalandı ve Accuracy ile karşılaşırıldı.

In [ ]:
clf =RandomForestClassifier(max_depth=15,random_state=42,criterion="entropy",n_estimators=900)
print(cross_val_score(clf, X, y, cv=10, n_jobs=1))

d)XGBoost Classifier

XGBoost Classifier modellinde kullanılacak parametreleri belirlemek için GridSearchCV ile en iyi score'u üretecek parametreler bulunur.

In [ ]:

param_grid = {'max_depth':[2, 4, 5, 7, 9, 10], 'learning_rate':[0.001, 0.01, 0.1, 0.2, 0.3], 'min_child_weight':[2, 4, 5, 6, 7]}

grid = GridSearchCV(XGBClassifier(), param_grid=param_grid, verbose=3)

grid.fit(X, y)
grid.best_params_

GridsearchCV ile bulunan en iyi parametreler XGBoost Classifier modeline parametre olarak uygulanr.Sınıflandırma raporlanır ve accuracy hesaplanır.

In [ ]:
xgb = XGBClassifier(learning_rate=0.3, max_depth=5, min_child_weight=2)
xgb.fit(X_train,y_train)
y_pred=xgb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
xgb.score(X_test,y_test)

XGBoost Classifier modeli için Cross Validation Score Hesalandı ve Accuracy ile karşılaşırıldı.

In [ ]:
clf = XGBClassifier(learning_rate=0.3, max_depth=5, min_child_weight=2)
print(cross_val_score(clf, X, y, cv=10, n_jobs=1))

SONUÇ



Proje kapsamında ilk olarak 'income' data seti incelendi data hakkında bilgi edinildi.Setteki verilerle görselleştirmeler yapıldı nümerik olmayan veriler nümerik formata dönüştürüldü.Dataya KNN,Logistic Regression,Random Forest ve XGBoost makine öğrenmesi modelleri uygulandı.Modeller uygulanmadan önce en iyi performans sağlayacağı parametreler GridSearch algoritması ile çıkartıldı ve bu parametrelerle çalıştırıldı sonuçlar raporlandI akabinde Cross Validation Score'ları ilede incelendi. Bu incelemeler sonucunda income kolonunda en fazla veri '0' yani <50K da olduğundan dolayı '0' ları öğrenme ve dolayısıyla tahminde '1' yani >50K'lardan daha iyi bir performans göstermiştir. Uygulanan modellerde en yüskek başarım XGBoost modelinde gözlemlenmiştir.


In [ ]:
print("KNN Model Score                  = ",knn.score(X_test,y_test))
print("Logistic Regression Score        = ",logmodel.score(X_test,y_test))
print("Random Forest Classifier Score   = ",randomforest.score(X_test,y_test))
print("XGBoost Forest Classifier Score  = ",xgb.score(X_test,y_test))